## Criação da Rede

In [34]:
# Dependências
from components import Network, Controller
from components.qkd.app import *
import numpy as np

In [35]:
# Criando a rede
rede = Network()
# Criando e definindo um controlador pra rede
controlador = Controller(rede)

In [36]:
# Propriedades para simulação
n_simulacoes = 100
n_requests = 10
n_total_execucoes = 0

## Requests

In [37]:
def generate_qkd_request(rede, num_requests , avaliable_apps=["BB84", "E91", "B92"], diff_nodes=5):
        """
        Gera uma lista de requisições aleatórias de QKD.

        Args:
            num_requests (int): Número de requisições.
            diff_nodes (int): Número entre os nós. Defauts to 5.

        Returns:
            requests (list): Lista com requisições.
        """
        requests = []
        
        for i in range(num_requests):
            alice, bob = rede.random_alice_bob(diff_nodes)
            priority = random.randint(1,5)
            app = random.choice(avaliable_apps)
            requests.append((alice, bob, app, priority))
        
        return requests

## Coleta dos resultados

In [38]:
# Taxa de sucesso de bits das chaves
taxas_sucesso_chaves_geral = [] 
taxas_sucesso_chaves_e91 = [] 
taxas_sucesso_chaves_bb84 = [] 
taxas_sucesso_chaves_b92 = [] 

# Vazão da rede
apps = ["BB84", "E91", "B92"]
vazao = [] 

In [39]:
# Executa num_simulacoes vezes simulações de requests
for indice, simulacao in enumerate(range(n_simulacoes)):
    
    taxas_sucesso_chaves_e91 = []
    taxas_sucesso_chaves_bb84 = []
    taxas_sucesso_chaves_b92 = []
    rede.set_lattice_topology(3, 4)

    requests = generate_qkd_request(rede, n_requests, apps)
    resultados_simulacao = controlador.send_requests(requests, 'shortest')
    
    for indice_execucao in resultados_simulacao:
        # Percorre cada execução e colhe o sucesso nas chaves
        resultado_individual_simulacao = resultados_simulacao[indice_execucao]
        sucesso_chave = resultado_individual_simulacao['key sucess']
        
        # Colhendo o sucesso nas chaves de cada app (Tamamnho da chave recebida/ Tamanho da chave enviada)
        if resultado_individual_simulacao['app'] == 'BB84':
            taxas_sucesso_chaves_bb84.append(sucesso_chave)
            media_sucesso_chaves_bb84 = sum(taxas_sucesso_chaves_bb84) / len(taxas_sucesso_chaves_bb84)
        
        elif resultado_individual_simulacao['app'] == 'E91':
            taxas_sucesso_chaves_e91.append(sucesso_chave)
            media_sucesso_chaves_e91 = sum(taxas_sucesso_chaves_e91) / len(taxas_sucesso_chaves_e91)
            
        elif resultado_individual_simulacao['app'] == 'B92':
            taxas_sucesso_chaves_b92.append(sucesso_chave)
            media_sucesso_chaves_b92 = sum(taxas_sucesso_chaves_b92) / len(taxas_sucesso_chaves_b92)
      
    
    # Salvando o sucesso nas chaves geral da simulação
    lista_combinada = [taxa for sublist in [taxas_sucesso_chaves_bb84, taxas_sucesso_chaves_e91, taxas_sucesso_chaves_b92] for taxa in sublist]
    #print(lista_combinada)
    taxas_sucesso_chaves_geral.append(sum(lista_combinada) / len(lista_combinada))
    
    # Calculando a vazão
    n_execucoes = len(resultados_simulacao)
    vazao.append(n_requests / n_execucoes)
    

In [40]:
def formatar_numero(numero):
    # Transforma o número em uma string com 7 casas decimais
    numero_formatado = "{:.7f}".format(numero)

    # Substitui o ponto pela vírgula
    numero_formatado = numero_formatado.replace('.', ',')

    return numero_formatado

In [41]:
print(formatar_numero(np.mean(vazao)))

5,6083333


In [42]:
print(formatar_numero(np.std(vazao)))

2,1533404


In [43]:
print(formatar_numero(np.mean(taxas_sucesso_chaves_geral)))

0,3835833


In [44]:
print(formatar_numero(np.std(taxas_sucesso_chaves_geral)))

0,1270742
